In [83]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import torch
from torch.utils.data import TensorDataset,DataLoader
import torch.nn.functional as F
import torch.nn as nn
import copy

def ML_pre(model_path, data_x):
    m = joblib.load(model_path)
    data_score = m.predict_proba(data_x)[:,1]
    data_pre = m.predict(data_x)
    data_out = data_x
    data_out['pre_score'] = data_score
    data_out['pre_y'] = data_pre
    return data_out

def DNN_prepare(data):
    data_copy = copy.deepcopy(data)
    x_df = data_copy
    x_arr = x_df.values
    x = torch.tensor(x_arr).float()
    dataset = TensorDataset(x)
    dataset_loader = DataLoader(dataset=dataset,batch_size=25,shuffle=False,num_workers=0)
    return dataset_loader

def decode(arr):
    l = []
    for i in range(len(arr)):
        if arr[i][1] > 0.5:
            l.append(1)
        elif arr[i][1] < 0.5:
            l.append(0)
        else:
            l.append('error')
    return l

def model_pred(best_model,loader,cuda=True,*device):
    with torch.no_grad():
        l_y_pre = []
        l_y_score = []
        for inputs in loader:
            if cuda == True:
                inputs = inputs.to(device[0])
            else:
                pass
            y_pre = best_model(inputs[0])
            if cuda == True:
                y_pre = y_pre.to('cpu')
            else:
                pass
            y_pre = y_pre.numpy().tolist()
            for i in range(len(y_pre)):
                l_y_score.append(y_pre[i][1])
            l_batch_pre = decode(y_pre)
            l_y_pre += l_batch_pre
    return l_y_pre, l_y_score

##network    
class MyNet(nn.Module):
    
    def __init__(self,inputs):
        super(MyNet,self).__init__()
        self.fc_1 = nn.Linear(inputs,120)
        self.bn_1 = nn.BatchNorm1d(120)
        self.fc_2 = nn.Linear(120,160)
        self.bn_2 = nn.BatchNorm1d(160)
        self.fc_3 = nn.Linear(160,120)
        self.bn_3 = nn.BatchNorm1d(120)
        self.fc_4 = nn.Linear(120,80)
        self.bn_4 = nn.BatchNorm1d(80)
        self.fc_5 = nn.Linear(80,20)
        self.bn_5 = nn.BatchNorm1d(20)
        self.fc_6 = nn.Linear(20,2)
        
    def forward(self,input_data):
        x = F.relu(self.bn_1(self.fc_1(input_data)))
        x = F.relu(self.bn_2(self.fc_2(x)))
        x = F.relu(self.bn_3(self.fc_3(x)))
        x = F.relu(self.bn_4(self.fc_4(x)))
        x = F.relu(self.bn_5(self.fc_5(x)))
        x = torch.sigmoid(self.fc_6(x))
        return x

In [87]:
## load model
model_path = '../model/Model2C.model'
pre_path = '../data/example_data_maccs_filter.csv'
df = pd.read_csv(pre_path)
if 'DNN' in model_path:
    data_out = df.copy()
    pre_loader = DNN_prepare(df)      
    best_net = torch.load(model_path)
    data_pre, data_score = model_pred(best_net, pre_loader, False)
    data_out['pre_score'] = data_score
    data_out['pre_y'] = data_pre
else:
    data_out = ML_pre(model_path, df)